# Welcome to our NASA Challenge

## Seismic Detection Across The Solar System

### ObsPy: A powerful library for reading and processing seismic data, especially .miniseed files.
### NumPy and Pandas: For handling data and doing calculations.
### Matplotlib and Seaborn: For plotting your results, such as the seismic waveforms and spectrograms.
### Sklearn : For Label Encoding, Train_Test_split, Random_Forest and Evaluation Metrics

# Importing Dependencies

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
import os
from obspy.signal.trigger import classic_sta_lta

ModuleNotFoundError: No module named 'obspy'

In [ ]:
# Read The CSV File into DataFrame 
lunar = pd.read_csv("apollo12_catalog_GradeA_final.csv")
lunar.head()

In [ ]:
lunar.info()

In [ ]:
# Convert 'time_abs(%Y-%m-%dT%H:%M:%S.%f)' to pandas datetime format
lunar['time_abs(%Y-%m-%dT%H:%M:%S.%f)'] = pd.to_datetime(lunar['time_abs(%Y-%m-%dT%H:%M:%S.%f)'])

lunar.head()

In [ ]:
# Plot the distribution of quake types
lunar['mq_type'].value_counts().plot(kind='bar', title="Distribution of Quake Types", figsize=(10, 6))
plt.xlabel("Quake Type")
plt.ylabel("Number of Events")
plt.show()

# First Solution [Logistic Regression & Random Forest Machine Learning Models]

In [ ]:
# Feature Engineering: Extract hour, day, month, etc., from Abs_Time
lunar['Hour'] = lunar['time_abs(%Y-%m-%dT%H:%M:%S.%f)'].dt.hour
lunar['Day'] = lunar['time_abs(%Y-%m-%dT%H:%M:%S.%f)'].dt.day
lunar['Month'] = lunar['time_abs(%Y-%m-%dT%H:%M:%S.%f)'].dt.month
lunar['Year'] = lunar['time_abs(%Y-%m-%dT%H:%M:%S.%f)'].dt.year

In [ ]:
# Encode the 'filename' as numeric using LabelEncoder
file_name_encoder = LabelEncoder()
lunar['File_Name_Encoded'] = file_name_encoder.fit_transform(lunar['filename'])

# Encode the target variable 'mq_type' as numeric
quake_type_encoder = LabelEncoder()
lunar['MQ_Type_Encoded'] = quake_type_encoder.fit_transform(lunar['mq_type'])

In [ ]:
lunar.info()

In [ ]:
lunar.head()

In [ ]:
# Drop columns that won't be used for classification (Abs_Time, File Name of Event, etc.)
features = lunar.drop(columns=['time_abs(%Y-%m-%dT%H:%M:%S.%f)', 'filename', 'mq_type', 'MQ_Type_Encoded', 'evid'])

# Define the target variable
target = lunar['MQ_Type_Encoded']

In [ ]:
# Split data into training and testing sets (80% training, 20% testing)
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Check the shape of the data to ensure it is ready for modeling
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

In [ ]:
# Create the model
rf_model = RandomForestClassifier(random_state=42)
logreg = LogisticRegression(random_state = 42)

# Train the model
rf_model.fit(X_train, y_train)
logreg.fit(X_train, y_train)

# Make predictions on the test set
y_pred_rf = rf_model.predict(X_test)
y_pred_lr = rf_model.predict(X_test)

# Evaluate the model
print("Random Forest")
print(f"Accuracy = {accuracy_score(y_test, y_pred_rf):.2f}%")
print(f"Classification Report:\n{classification_report(y_test, y_pred_rf)}")
print(f"Confusion Matrix\n{confusion_matrix(y_test, y_pred_rf)}")

print("="*50)

print("Logistic Regression")
print(f"Accuracy = {accuracy_score(y_test, y_pred_lr):.2f}%")
print(f"Classification Report:\n{classification_report(y_test, y_pred_lr)}")
print(f"Confusion Matrix\n{confusion_matrix(y_test, y_pred_lr)}")

In [ ]:
new_data = X_test.iloc[0:5]

# Predict the Quake_Type
predicted_quake_type_rf = rf_model.predict(new_data)
predicted_quake_type_lr = logreg.predict(new_data)

# Decode the predicted labels back to their original quake type names
predicted_quake_type_labels = quake_type_encoder.inverse_transform(predicted_quake_type_rf)
print(predicted_quake_type_labels)

# Decode the predicted labels back to their original quake type names
predicted_quake_type_labels = quake_type_encoder.inverse_transform(predicted_quake_type_lr)
print(predicted_quake_type_labels)

In [ ]:
# Create a confusion matrix to visualize the predictions
conf_matrix = confusion_matrix(y_test, y_pred_rf)

plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=quake_type_encoder.classes_, yticklabels=quake_type_encoder.classes_)
plt.title('Confusion Matrix [Random Forest]')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

In [ ]:
# Create a confusion matrix to visualize the predictions for Logistic Regression
conf_matrix = confusion_matrix(y_test, y_pred_lr)

plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=quake_type_encoder.classes_, yticklabels=quake_type_encoder.classes_)
plt.title('Confusion Matrix [Logistic Regression]')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

# Second Solution [Applying STA/LTA Algorithm (Apply a Threshold) ]

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Extract the seismic signal (using Rel_Time as an example)
signal = lunar['time_rel(sec)'].values

# Define the STA/LTA function
def sta_lta(signal, sta_len, lta_len):
    sta = np.convolve(signal, np.ones(sta_len) / sta_len, mode='same')
    lta = np.convolve(signal, np.ones(lta_len) / lta_len, mode='same')
    lta[lta == 0] = np.nan
    sta_lta_ratio = sta / lta
    return sta, lta, sta_lta_ratio

# Set lengths for STA and LTA
sta_length = 5
lta_length = 50

# Calculate STA, LTA, and STA/LTA ratio
sta, lta, sta_lta_ratio = sta_lta(signal, sta_length, lta_length)

# Define a threshold for detection
threshold = 1.5
detected_events = sta_lta_ratio > threshold

# Print and save detected event times
event_times = lunar['time_abs(%Y-%m-%dT%H:%M:%S.%f)'][detected_events]
event_times_rel = lunar['time_rel(sec)'][detected_events]
event_files = lunar["filename"][detected_events]

# Create DataFrame for detected events
df_events = pd.DataFrame({
    "File_Name": event_files,
    "Abs_Time": event_times,
    "Rel_Time": event_times_rel,
    "STA": sta[detected_events],
    "LTA": lta[detected_events],
    "STA/LTA Ratio": sta_lta_ratio[detected_events]
})

# Save STA/LTA data to CSV
output_file = 'detected_events_sta_lta.csv'
df_events.to_csv(output_file, index=False)

# Visualize the results
plt.figure(figsize=(15, 6))
plt.plot(sta_lta_ratio, label='STA/LTA Ratio', color='blue')
plt.axhline(threshold, color='red', linestyle='--', label='Threshold')
plt.scatter(np.arange(len(sta_lta_ratio))[detected_events], sta_lta_ratio[detected_events], color='green', label='Detected Events')
plt.title('STA/LTA Ratio and Detected Events')
plt.xlabel('Samples')
plt.ylabel('STA/LTA Ratio')
plt.legend()
plt.show()

print(f"Saved STA/LTA data to {output_file}")


In [ ]:
def sta_lta(signal, sta_len, lta_len):
    # Calculate short-term average (STA)
    sta = np.convolve(signal, np.ones(sta_len) / sta_len, mode='same')

    # Calculate long-term average (LTA)
    lta = np.convolve(signal, np.ones(lta_len) / lta_len, mode='same')

    # Ensure both STA and LTA have the same length by using the shorter one
    min_len = min(len(sta), len(lta))
    sta = sta[:min_len]
    lta = lta[:min_len]
    
    # Avoid division by zero in LTA
    lta[lta == 0] = np.nan
    
    # Calculate STA/LTA ratio
    sta_lta_ratio = sta / lta
    
    return sta, lta, sta_lta_ratio


In [ ]:
for sta_length in sta_lengths:
    for lta_length in lta_lengths:
        for threshold in thresholds:
            # Calculate STA/LTA
            sta, lta, sta_lta_ratio = sta_lta(signal, sta_length, lta_length)
            
            # Detect events
            detected_events = sta_lta_ratio > threshold
            event_times = lunar['time_abs(%Y-%m-%dT%H:%M:%S.%f)'][detected_events]
            event_files = lunar["filename"][detected_events]
            event_times_rel = lunar['time_rel(sec)'][detected_events]

            # Create DataFrame for detected events
            df_events = pd.DataFrame({
                "File_Name": event_files,
                "Abs_Time": event_times,
                "Rel_Time": event_times_rel
            })

            # Match detected events with ground truth
            TP, FP, FN = match_events(ground_truth, df_events, pd.Timedelta(seconds=10))
            
            # Calculate accuracy metrics
            accuracy = TP / (TP + FP + FN) if (TP + FP + FN) > 0 else 0
            precision = TP / (TP + FP) if (TP + FP) > 0 else 0
            recall = TP / (TP + FN) if (TP + FN) > 0 else 0
            f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

            # Check if this configuration is the best so far
            if accuracy > best_accuracy:
                best_accuracy = accuracy
                best_params = {
                    'sta_length': sta_length,
                    'lta_length': lta_length,
                    'threshold': threshold,
                    'accuracy': accuracy,
                    'precision': precision,
                    'recall': recall,
                    'f1_score': f1_score
                }


In [ ]:
print(accuracy)

In [ ]:
# Print the detected event times
event_times = lunar['time_abs(%Y-%m-%dT%H:%M:%S.%f)'][detected_events]
event_times_rel = lunar['time_rel(sec)'][detected_events]
event_files = lunar["filename"][detected_events]

df = pd.DataFrame({
    "File_Name" : event_files,
    "Abs_Time" : event_times,
    "Rel_Time" : event_times_rel
})
df

# Anthor Third Solution By Machine Learning Model with [STA, LTA, STA/LTA] Features

In [ ]:
# Extract the signal (assuming Rel_Time is a numeric signal)
signal = lunar['time_rel(sec)'].values

In [ ]:
# Calculate STA, LTA, and STA/LTA ratio
sta_length = 5
lta_length = 50
sta, lta, sta_lta_ratio = sta_lta(signal, sta_length, lta_length)

In [ ]:
# Add features to the DataFrame
lunar['STA'] = sta
lunar['LTA'] = lta
lunar['STA/LTA'] = sta_lta_ratio

In [ ]:
lunar.head()

In [ ]:
lunar.shape

In [ ]:
lunar[lunar["STA/LTA"] > 1]

In [ ]:
lunar.info()

In [ ]:
lunar.describe().T

In [ ]:
# Create the target variable (assuming Quake_Type is your label)
X = lunar[['STA', 'LTA', 'STA/LTA']]  # Features
y = lunar['MQ_Type_Encoded']  # Labels

In [ ]:
# Step 7: Train/Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Step 8: Train the Random Forest Classifier
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

In [ ]:
# Make Predictions
y_pred = model.predict(X_test)

# Evaluate the Model
print(f"Accuracy = {accuracy_score(y_test, y_pred) * 100 :.2F} %")
print(f"Confusion Matrix:\n{confusion_matrix(y_test, y_pred)}")
print(f"\nClassification Report:\n{classification_report(y_test, y_pred)}")

In [ ]:
# Plot the STA/LTA Ratio and Detected Events
plt.figure(figsize=(15, 6))

# Plot STA/LTA ratio
plt.plot(sta_lta_ratio, label='STA/LTA Ratio', color='blue')
plt.axhline(1.5, color='red', linestyle='--', label='Threshold')  # Change threshold if needed

# Highlight detected events based on the threshold
detected_events = sta_lta_ratio > 1.5
plt.scatter(np.arange(len(sta_lta_ratio))[detected_events], sta_lta_ratio[detected_events], color='green', label='Detected Events')

plt.title('STA/LTA Ratio and Detected Events')
plt.xlabel('Samples')
plt.ylabel('STA/LTA Ratio')
plt.legend()
plt.show()

# Testing

## Combine all Test Files into one file

In [ ]:
import os
import pandas as pd

# Directory containing the CSV files
directory = 'D:\\NASA Challenge\\space_apps_2024_seismic_detection\\data\\lunar\\test\\data\\S12_GradeB\\'
combined_data = []

for file in os.listdir(directory):
    if file.endswith('.csv'):
        # Try reading the CSV file using engine='python'
        try:
            df = pd.read_csv(os.path.join(directory, file), engine='python')
            combined_data.append(df)
        except pd.errors.ParserError as e:
            print(f"Error parsing {file}: {e}")

# Combine all dataframes into one
if combined_data:
    combined_df = pd.concat(combined_data, ignore_index=True)
    print("Successfully combined all CSV files.")
else:
    print("No valid CSV files to combine.")


In [ ]:
S12_GradeB.head()

### Test on File [S12_GradeB]

In [ ]:
signal = S12_GradeB['time_rel(sec)'].values

In [ ]:
# Set lengths for STA and LTA
sta_length = 5    # Short-term average window length
lta_length = 50   # Long-term average window length

# Calculate STA, LTA, and STA/LTA ratio
sta, lta, sta_lta_ratio = sta_lta(signal, sta_length, lta_length)

In [ ]:
# Define a threshold for detection
threshold = 1.5

# Create an array for event detection based on the threshold
detected_events = sta_lta_ratio > threshold

In [ ]:
# Print the detected event times
event_times = S12_GradeB['time_abs(%Y-%m-%dT%H:%M:%S.%f)'][detected_events]
print(f"Detected Events:\n{event_times}")

In [ ]:
# Print the detected event times
event_times = S12_GradeB['time_abs(%Y-%m-%dT%H:%M:%S.%f)'][detected_events]
event_times_rel = S12_GradeB['time_rel(sec)'][detected_events]

df = pd.DataFrame({
    "Abs_Time" : event_times,
    "Rel_Time" : event_times_rel
})
df

In [ ]:
# Step 6: Visualize the results
plt.figure(figsize=(25, 6))

# Plot STA/LTA ratio
plt.plot(sta_lta_ratio, label='STA/LTA Ratio', color='blue')
plt.axhline(threshold, color='red', linestyle='--', label='Threshold')

# Highlight detected events
plt.scatter(np.arange(len(sta_lta_ratio))[detected_events], sta_lta_ratio[detected_events], color='green', label='Detected Events')

plt.title('STA/LTA Ratio and Detected Events')
plt.xlabel('Samples')
plt.ylabel('STA/LTA Ratio')
plt.legend()
plt.show()

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from obspy.signal.trigger import classic_sta_lta, trigger_onset

def apply_sta_lta(csv_file, sta_len=120, lta_len=600, sampling_rate=100, threshold=2.0):
    """
    Function to read a CSV file, apply STA/LTA, and plot the characteristic function with detected events.
    
    Parameters:
    csv_file (str): Path to the CSV file.
    sta_len (int): Length of the short-term window in seconds.
    lta_len (int): Length of the long-term window in seconds.
    sampling_rate (int): Sampling frequency of the data in Hz.
    threshold (float): STA/LTA threshold for event detection.
    
    Returns:
    None
    """
    
    # Step 1: Read the CSV file
    data = pd.read_csv(csv_file)
    
    # Check if required columns are present in the CSV file
    if 'time_rel(sec)' not in data.columns or 'velocity(m/s)' not in data.columns:
        raise ValueError(f"CSV file {csv_file} must contain 'time_rel(sec)' and 'velocity(m/s)' columns.")
    
    # Step 2: Extract relevant columns
    rel_time = data['time_rel(sec)'].values  # Time data (X-axis)
    velocity = data['velocity(m/s)'].values  # Seismic amplitude or velocity (Y-axis)
    
    # Step 3: Calculate STA/LTA using Obspy's function
    sta_samples = int(sta_len * sampling_rate)  # Convert seconds to samples
    lta_samples = int(lta_len * sampling_rate)  # Convert seconds to samples
    
    # Apply classic STA/LTA to the velocity data
    cft = classic_sta_lta(velocity, sta_samples, lta_samples)
    
    # Step 4: Detect events based on the STA/LTA characteristic function
    onsets = trigger_onset(cft, threshold, threshold / 2)  # Get onset and end times of events
    
    # Step 5: Plot the characteristic function (STA/LTA ratio over time)
    plt.figure(figsize=(12, 5))
    plt.plot(rel_time, cft, label='STA/LTA Characteristic Function')
    plt.axhline(y=threshold, color='red', linestyle='--', label='Threshold')
    
    # Mark detected events on the plot
    for onset in onsets:
        start_time = rel_time[onset[0]]
        plt.plot(start_time, threshold, 'go', markersize=10, label='Detected Event')  # Marker for detected event
    
    plt.xlabel('Relative Time (s)')
    plt.ylabel('STA/LTA Ratio')
    plt.title('STA/LTA Characteristic Function with Detected Events')
    plt.xlim([min(rel_time), max(rel_time)])
    plt.legend()
    plt.grid(True)
    plt.show()

# Example usage
csv_file_path = 'S12_GradeB.csv'  # Replace with the actual path to your CSV file
apply_sta_lta(csv_file_path, sta_len=120, lta_len=600, sampling_rate=100)

import os

# Directory containing test CSV files
test_dir = 'D:\\NASA Challenge\\space_apps_2024_seismic_detection\\data\\lunar\\test\\data\\S12_GradeB\\'  

# Loop through each file in the directory and apply STA/LTA
for file_name in os.listdir(test_dir):
    file_path = os.path.join(test_dir, file_name)
    print(f"Processing file: {file_name}")

    try:
        apply_sta_lta(file_path, sta_len=120, lta_len=600, sampling_rate=100)
    except Exception as e:
        print(f"Error processing {file_name}: {e}")



In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from obspy.signal.trigger import classic_sta_lta, trigger_onset

def apply_sta_lta(csv_file, sta_len=120, lta_len=600, sampling_rate=100, threshold=2.0):
    """
    Function to read a CSV file, apply STA/LTA, detect events, and output results.
    
    Parameters:
    csv_file (str): Path to the CSV file.
    sta_len (int): Length of the short-term window in seconds.
    lta_len (int): Length of the long-term window in seconds.
    sampling_rate (int): Sampling frequency of the data in Hz.
    threshold (float): STA/LTA threshold for event detection.
    
    Returns:
    pd.DataFrame: DataFrame with detected events.
    """
    
    # Step 1: Read the CSV file
    data = pd.read_csv(csv_file)
    
    # Check if required columns are present in the CSV file
    if 'time_rel(sec)' not in data.columns or 'velocity(m/s)' not in data.columns:
        raise ValueError(f"CSV file {csv_file} must contain 'Rel_Time' and 'Velocity' columns.")
    
    # Step 2: Extract relevant columns
    rel_time = data['time_rel(sec)'].values  # Time data (X-axis)
    velocity = data['velocity(m/s)'].values  # Seismic amplitude or velocity (Y-axis)
    
    # Step 3: Calculate STA/LTA using Obspy's function
    sta_samples = int(sta_len * sampling_rate)  # Convert seconds to samples
    lta_samples = int(lta_len * sampling_rate)  # Convert seconds to samples
    
    # Apply classic STA/LTA to the velocity data
    cft = classic_sta_lta(velocity, sta_samples, lta_samples)
    
    # Step 4: Detect events based on the STA/LTA characteristic function
    onsets = trigger_onset(cft, threshold, threshold / 2)  # Get onset and end times of events
    
    # Step 5: Prepare output data for detected events
    event_data = []
    for onset in onsets:
        start_time = rel_time[onset[0]]
        event_data.append({
            'filename': csv_file,
            'time_rel': start_time  # You can adjust this to include absolute time if available
        })
    
    # Create a DataFrame for the output catalog
    output_df = pd.DataFrame(event_data)
    
    # Save the output catalog to a CSV file
    output_file = f"detected_events_{csv_file.split('/')[-1].replace('.csv', '')}.csv"
    output_df.to_csv(output_file, index=False)
    
    # Plotting characteristic function and detected events
    plt.figure(figsize=(12, 5))
    plt.plot(rel_time, cft, label='STA/LTA Characteristic Function')
    plt.axhline(y=threshold, color='red', linestyle='--', label='Threshold')
    
    # Mark detected events on the plot
    for onset in onsets:
        start_time = rel_time[onset[0]]
        plt.plot(start_time, threshold, 'go', markersize=10)  # Marker for detected event
    
    plt.xlabel('Relative Time (s)')
    plt.ylabel('STA/LTA Ratio')
    plt.title('STA/LTA Characteristic Function with Detected Events')
    plt.xlim([min(rel_time), max(rel_time)])
    plt.legend()
    plt.grid(True)
    plt.show()
    
    return output_df  # Return the DataFrame for further use if needed

# Example usage
csv_file_path = 'S12_gradeB.csv'  # Replace with the actual path to your CSV file
output_df = apply_sta_lta(csv_file_path, sta_len=120, lta_len=600, sampling_rate=100, threshold=2.0)
output_df

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
from obspy.signal.trigger import classic_sta_lta, trigger_onset
from scipy.signal import butter, filtfilt

def highpass_filter(data, cutoff, fs, order=4):
    nyquist = 0.5 * fs
    normal_cutoff = cutoff / nyquist
    b, a = butter(order, normal_cutoff, btype='high', analog=False)
    filtered_data = filtfilt(b, a, data)
    return filtered_data

def apply_sta_lta(csv_file, sta_len=120, lta_len=600, sampling_rate=100, threshold=2.0):
    # Step 1: Read the CSV file
    data = pd.read_csv(csv_file)
    
    # Step 2: Extract relevant columns
    rel_time = data['time_rel(sec)'].values  # Time data (X-axis)
    velocity = data['velocity(m/s)'].values  # Seismic amplitude or velocity (Y-axis)
    
    # Step 3: Preprocess the data (highpass filtering)
    velocity_filtered = highpass_filter(velocity, cutoff=1.0, fs=sampling_rate)
    
    # Step 4: Calculate STA/LTA
    sta_samples = int(sta_len * sampling_rate)
    lta_samples = int(lta_len * sampling_rate)
    cft = classic_sta_lta(velocity_filtered, sta_samples, lta_samples)
    
    # Step 5: Detect events based on threshold
    onsets = trigger_onset(cft, threshold, threshold / 2)
    
    # Step 6: Plot the STA/LTA function
    plt.figure(figsize=(12, 5))
    plt.plot(rel_time, cft, label='STA/LTA Characteristic Function')
    plt.axhline(threshold, color='r', linestyle='--', label='Threshold')
    
    # Mark detected quake events
    for onset in onsets:
        plt.axvline(rel_time[onset[0]], color='g', linestyle='--', label='Quake Detected')
    
    plt.xlabel('Relative Time (s)')
    plt.ylabel('STA/LTA Ratio')
    plt.title('STA/LTA Characteristic Function with Quake Detection')
    plt.xlim([min(rel_time), max(rel_time)])
    plt.legend()
    plt.grid(True)
    plt.show()

# Example usage
csv_file_path = 'S12_GradeB.csv'  # Replace with actual CSV path
apply_sta_lta(csv_file_path, sta_len=120, lta_len=600, sampling_rate=100, threshold=2.0)


### Test on Mars Data

In [ ]:
# Specify the directory where the CSV files are located
directory = 'D:\\NASA Challenge\\space_apps_2024_seismic_detection\\data\\mars\\test\\data\\'

# Create an empty list to hold the dataframes
combined_data = []

# Loop through all files in the directory
for file in os.listdir(directory):
    if file.endswith('.csv'):
        # Read each CSV file
        df = pd.read_csv(os.path.join(directory, file))
        combined_data.append(df)

# Combine all dataframes into one
Mars = pd.concat(combined_data, ignore_index=True)

# Save the combined dataframe to a new CSV file
Mars.to_csv('Mars Test.csv', index=False)

print("Files combined successfully!")

In [ ]:
Mars.head()

In [ ]:
Mars["velocity(m/s)"] = Mars["velocity(c/s)"] / 100

In [ ]:
Mars.head()

In [ ]:
# Extract the signal (assuming Rel_Time is a numeric signal)
signal = Mars['rel_time(sec)'].values

In [ ]:
# Set lengths for STA and LTA
sta_length = 5    # Short-term average window length
lta_length = 50   # Long-term average window length

# Calculate STA, LTA, and STA/LTA ratio
sta, lta, sta_lta_ratio = sta_lta(signal, sta_length, lta_length)

# Define a threshold for detection
threshold = 1.5

# Create an array for event detection based on the threshold
detected_events = sta_lta_ratio > threshold

In [ ]:
# Print the detected event times
event_times = Mars['time(%Y-%m-%dT%H:%M:%S.%f)'][detected_events]
print(f"Detected Events:\n{event_times}")

In [ ]:
# Print the detected event times
event_times = Mars['time(%Y-%m-%dT%H:%M:%S.%f)'][detected_events]
event_times_rel = Mars['rel_time(sec)'][detected_events]

df = pd.DataFrame({
    "Abs_Time" : event_times,
    "Rel_Time" : event_times_rel
})
df

In [ ]:
# Visualize the results
plt.figure(figsize=(15, 6))

# Plot STA/LTA ratio
plt.plot(sta_lta_ratio, label='STA/LTA Ratio', color='blue')
plt.axhline(threshold, color='red', linestyle='--', label='Threshold')

# Highlight detected events
plt.scatter(np.arange(len(sta_lta_ratio))[detected_events], sta_lta_ratio[detected_events], color='green', label='Detected Events')

plt.title('STA/LTA Ratio and Detected Events')
plt.xlabel('Samples')
plt.ylabel('STA/LTA Ratio')
plt.legend()
plt.show()